### Author: Hoang Chu

### Imports

In [1]:
%pip install -r dependencies.txt

INFO: pip is looking at multiple versions of qtpy to determine which version is compatible with other requirements. This could take a while.
  Using cached QtPy-2.3.0-py3-none-any.whl (83 kB)
INFO: pip is looking at multiple versions of qtconsole to determine which version is compatible with other requirements. This could take a while.
  Using cached qtconsole-5.4.0-py3-none-any.whl (121 kB)
INFO: pip is looking at multiple versions of pyzmq to determine which version is compatible with other requirements. This could take a while.
  Using cached pyzmq-25.0.0-cp310-cp310-macosx_10_15_universal2.whl (1.8 MB)
INFO: pip is looking at multiple versions of pyyaml to determine which version is compatible with other requirements. This could take a while.
  Using cached PyYAML-6.0-cp310-cp310-macosx_11_0_arm64.whl (173 kB)
INFO: pip is looking at multiple versions of pytz to determine which version is compatible with other requirements. This could take a while.
  Using cached pytz-2023.3-py2.py

In [2]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import requests
import scipy
import importlib
import helper, consts
importlib.reload(consts)
importlib.reload(helper)
import glob

/opt/homebrew/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.5) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


### CONSTANTS

In [3]:
ROW = consts.ROW
COL = consts.COL
RAW_DATA_PATH = consts.RAW_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

RESPONSE_NAME

'tonight'

### Quick Functions

In [4]:
does_sub_df_has_Nan = lambda df, col_name: df[[col_name]].isna().any(axis=COL)

### Read data

In [7]:
# local read

# take in testing day - split into year and month 

#trainRange = helper.get_train_from_testday('20150102')
#files = helper.get_file_names(trainRange[0],trainRange[1]) # get filenames from before 6/1/2015

files = helper.get_file_names('20150101','20150601')
dfs = [pd.read_csv(consts.DATA_PATH_2015 + f"{f}") for f in files]
main_df = pd.concat(dfs)
main_df.shape
test_df = pd.read_csv(consts.DATA_PATH_2015 + "data.20150601_1200.csv")
x_cols = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0","rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0"]
saved_cols = x_cols + [RESPONSE_NAME]
training_df = main_df[saved_cols]
testing_df = test_df[saved_cols]

# folder_path = '../data/training_data'
# file_list = glob.glob(folder_path + "/*.csv") 
# main_df = pd.DataFrame(pd.read_csv(file_list[0])) 
# for i in range(1,len(file_list)): 
#     df = pd.read_csv(file_list[i]) 
#     # df = pd.DataFrame(data) 
#     main_df = pd.concat([main_df,df],axis=0) 

# df = pd.concat(map(pd.read_csv,glob.glob(folder_path + '/*.csv')))

#data_20191202_1200_df = pd.read_csv(RAW_DATA_PATH + "data_20191202_1200.csv")
#data_20191203_1200_df = pd.read_csv(RAW_DATA_PATH + "data_20191203_1200.csv")

In [8]:
main_df.shape


(309918, 252)

In [12]:
# predicting df

data_20150602_1200_df = pd.read_csv(consts.DATA_PATH_2015 + "data.20150602_1200.csv")

### Feature Selection

##### Preliminary check

In [13]:
main_df.shape

(309918, 252)

In [14]:
data_20150602_1200_df.shape

(3146, 252)

In [15]:
print(f"20191202:")
main_df.info()
print(f"Columns with NaN: {main_df[main_df.isna().any(axis=COL)]}")

print()

print(f"20191203:")
data_20150602_1200_df.info()
print(f"Columns with NaN: {data_20150602_1200_df[data_20150602_1200_df.isna().any(axis=COL)]}")

20191202:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 309918 entries, 0 to 3145
Columns: 252 entries, eqid to tr_toxhr_0
dtypes: float64(251), int64(1)
memory usage: 598.2 MB
Columns with NaN: Empty DataFrame
Columns: [eqid, wt, today, tonight, tmwam, dn3sttmwmd, dn1, dn4x1, cftorrrelstd_open_0, liqlog_open_0, llirpnxm_am_1, llirpnxm_dy_12to16, llirpnxm_dy_17to21, llirpnxm_dy_1to3, llirpnxm_dy_4to6, llirpnxm_dy_7to11, llirpnxm_lst15_0, llirpnxm_lsthrx15_0, llirpnxm_md_1, llirpnxm_nt_0, llirpnxm_nt_1, llirpnxm_nt_12to16, llirpnxm_nt_17to21, llirpnxm_nt_1to3, llirpnxm_nt_4to6, llirpnxm_nt_7to11, llirpnxm_pm_1, llirpnxm_toxhr_0, mocrelvol_open_1, mocrelvol_open_13to17, mocrelvol_open_18to22, mocrelvol_open_2to4, mocrelvol_open_5to7, mocrelvol_open_8to12, momr10d_open_0, momr50dx10d_open_0, moorelvol_open_0, moorelvol_open_12to16, moorelvol_open_17to21, moorelvol_open_1to3, moorelvol_open_4to6, moorelvol_open_7to11, nnetticksrelmultstdev_am_1, nnetticksrelmultstdev_dy_12to16, nnettic

In [16]:
print(f"20191202 - unique eqid count: {main_df.eqid.nunique()}")
print(f"20191203 - unique eqid count: {data_20150602_1200_df.eqid.nunique()}")

20191202 - unique eqid count: 3374
20191203 - unique eqid count: 3146


In [17]:
main_df.drop('eqid', axis=COL, inplace=True)
data_20150602_1200_df.drop('eqid', axis=COL, inplace=True)

In [18]:
print(f"training set:")
main_df.describe().loc[['min', 'max']]

training set:


,wt,today,tonight,tmwam,dn3sttmwmd,dn1,dn4x1,cftorrrelstd_open_0,liqlog_open_0,llirpnxm_am_1,...,tr_md_1,tr_nt_0,tr_nt_1,tr_nt_12to16,tr_nt_17to21,tr_nt_1to3,tr_nt_4to6,tr_nt_7to11,tr_pm_1,tr_toxhr_0
min,0.000015,-0.093743,-0.118355,-0.082232,-0.406343,-0.217713,-0.413634,-2.029335,-3.422019,-0.030027,...,-0.181246,-0.248395,-0.248395,-0.262104,-0.262104,-0.248395,-0.248395,-0.262104,-0.160350,-0.198276
max,0.003904,0.102819,0.151392,0.092196,0.325323,0.215648,0.331000,2.498018,1.810317,0.031884,...,0.249656,0.364367,0.364367,0.527026,0.527026,0.511446,0.511446,0.527026,0.289527,0.236002


In [19]:
print(f"20150602:")
data_20150602_1200_df.describe().loc[['min', 'max']]

20150602:


,wt,today,tonight,tmwam,dn3sttmwmd,dn1,dn4x1,cftorrrelstd_open_0,liqlog_open_0,llirpnxm_am_1,...,tr_md_1,tr_nt_0,tr_nt_1,tr_nt_12to16,tr_nt_17to21,tr_nt_1to3,tr_nt_4to6,tr_nt_7to11,tr_pm_1,tr_toxhr_0
min,0.000016,-0.045065,-0.052672,-0.042489,-0.203398,-0.099692,-0.2117,-1.606556,-3.331233,-0.007814,...,-0.064435,-0.072432,-0.218679,-0.242618,-0.236863,-0.218679,-0.185715,-0.151686,-0.023336,-0.079471
max,0.001342,0.057321,0.066898,0.069924,0.223958,0.164918,0.2377,2.501099,1.580095,0.012865,...,0.120576,0.053742,0.277050,0.235876,0.267265,0.532310,0.176683,0.364367,0.073329,0.077337


In [20]:
RESPONSE_NAME

'tonight'

##### Get the responses columns

In [21]:
training_responses = main_df[RESPONSE_NAME]
testing_responses = data_20150602_1200_df[RESPONSE_NAME]

##### 20191202_1200 data

In [22]:
main_df.drop(RESPONSE_NAME, axis=COL, inplace=True)

In [23]:
main_df.corr().isna().sum()

wt             7
today          7
tmwam          7
dn3sttmwmd     7
dn1            7
              ..
tr_nt_1to3     7
tr_nt_4to6     7
tr_nt_7to11    7
tr_pm_1        7
tr_toxhr_0     7
Length: 250, dtype: int64

Some column pairs have NaN correlations, meaning either or both columns have constant values in all rows.

In [24]:
# Get columns where all rows have the same value
constVal_columns = main_df.columns[main_df.apply(lambda x: x.nunique() == 1)]
constVal_columns

Index(['rvdelta_nt_0', 'rvdelta_nt_1', 'rvdelta_nt_12to16',
       'rvdelta_nt_17to21', 'rvdelta_nt_1to3', 'rvdelta_nt_4to6',
       'rvdelta_nt_7to11'],
      dtype='object')

In [25]:
TOBE_REMOVED_20191202_1200_COLUMNS = list(constVal_columns)

##### 20191203_1200 data

In [26]:
data_20150602_1200_df.drop(RESPONSE_NAME, axis=COL, inplace=True)

In [27]:
data_20150602_1200_df.corr().isna().sum()

wt             7
today          7
tmwam          7
dn3sttmwmd     7
dn1            7
              ..
tr_nt_1to3     7
tr_nt_4to6     7
tr_nt_7to11    7
tr_pm_1        7
tr_toxhr_0     7
Length: 250, dtype: int64

In [28]:
# Get columns where all rows have the same value
constVal_columns = data_20150602_1200_df.columns[data_20150602_1200_df.apply(lambda x: x.nunique() == 1)]
constVal_columns

Index(['rvdelta_nt_0', 'rvdelta_nt_1', 'rvdelta_nt_12to16',
       'rvdelta_nt_17to21', 'rvdelta_nt_1to3', 'rvdelta_nt_4to6',
       'rvdelta_nt_7to11'],
      dtype='object')

In [29]:
TOBE_REMOVED_20191203_1200_COLUMNS = list(constVal_columns)

##### Get training_predictors_df and testing_predictors_df

In [30]:
TOBE_REMOVED_20191202_1200_COLUMNS_set = set(TOBE_REMOVED_20191202_1200_COLUMNS)
TOBE_REMOVED_20191203_1200_COLUMNS_set = set(TOBE_REMOVED_20191203_1200_COLUMNS)

In [31]:
# Appear in TOBE_REMOVED_20191202_1200_COLUMNS_set but not in TOBE_REMOVED_20191203_1200_COLUMNS_set
print(TOBE_REMOVED_20191202_1200_COLUMNS_set - TOBE_REMOVED_20191203_1200_COLUMNS_set)

set()


In [32]:
# Appear in TOBE_REMOVED_20191203_1200_COLUMNS_set but not in TOBE_REMOVED_20191202_1200_COLUMNS_set
print(TOBE_REMOVED_20191203_1200_COLUMNS_set - TOBE_REMOVED_20191202_1200_COLUMNS_set)

set()


In [33]:
remove_20191202_stay_20191203_columns = list(TOBE_REMOVED_20191202_1200_COLUMNS_set - TOBE_REMOVED_20191203_1200_COLUMNS_set)
data_20150602_1200_df[remove_20191202_stay_20191203_columns].describe()

ValueError: Cannot describe a DataFrame without columns

In [ ]:
TOBE_REMOVED_COLUMNS = TOBE_REMOVED_20191202_1200_COLUMNS

Since there are variances among those columns, I don't think removing them now benefits the model.

In [34]:
data_20191202_1200_working_df = main_df.drop(TOBE_REMOVED_COLUMNS, axis=COL)
data_20191203_1200_working_df = data_20150602_1200_df.drop(TOBE_REMOVED_COLUMNS, axis=COL)

NameError: name 'TOBE_REMOVED_COLUMNS' is not defined

In [35]:
'rrirpnxm_nt_0' in main_df.columns

True

Rosy: we will not remove the columns right now from above, instead, we will just select the predictor columns we want to use

In [36]:
trainig_working_df =  main_df[['rrirpnxm_nt_0','rrirpnxm_lst15_0','rrirpnxm_toxhr_0','rrirpnxm_lsthrx15_0']].copy()
testing_working_df = data_20150602_1200_df[['rrirpnxm_nt_0','rrirpnxm_lst15_0','rrirpnxm_toxhr_0','rrirpnxm_lsthrx15_0']].copy()


In [37]:
print(f"1% negative correlations ranking: {sorted(trainig_working_df.corr().quantile(0.1), reverse=True)}")
print(f"99% negative correlations ranking: {sorted(trainig_working_df.corr().quantile(0.9), reverse=True)}")
trainig_working_df.corr().describe()

1% negative correlations ranking: [-0.025984035108515352, -0.04010837969125543, -0.04451972163578744, -0.057638337420823915]
99% negative correlations ranking: [0.700494493689765, 0.700494493689765, 0.7001372889622377, 0.6969260714691754]


,rrirpnxm_nt_0,rrirpnxm_lst15_0,rrirpnxm_toxhr_0,rrirpnxm_lsthrx15_0
count,4.000000,4.000000,4.000000,4.000000
mean,0.234578,0.239668,0.220672,0.231114
std,0.511197,0.507088,0.520020,0.512929
min,-0.063796,-0.032729,-0.063796,-0.043271
25%,-0.015606,-0.015867,-0.048402,-0.035364
50%,0.001053,-0.004299,-0.026759,-0.016136
75%,0.251236,0.251236,0.242315,0.250343
max,1.000000,1.000000,1.000000,1.000000


In [38]:
# Box to do figure out which columns to use
training_predictors_df = data_20191202_1200_working_df.copy()
testing_predictors_df = data_20191203_1200_working_df.copy()

NameError: name 'data_20191202_1200_working_df' is not defined

In [ ]:
sorted(training_predictors_df.quantile(0.25) / training_predictors_df.quantile(0.01), reverse=True)[:2]

In [ ]:
sorted(training_predictors_df.quantile(0.99) / training_predictors_df.quantile(0.75), reverse=True)[:2]

In [ ]:
plt.hist(training_responses, bins=100)

#### Detect influential points

I will use Mahalanobis distance, which computes distances from the data's centre and create an oval boundary.

In [ ]:
mean_vector = np.mean(training_predictors_df, axis=0)
cov_matrix = np.cov(training_predictors_df, rowvar=False)
inv_cov_matrix = np.linalg.inv(cov_matrix)

In [ ]:
mahalanobis_distances = []
for index, row in training_predictors_df.iterrows():
    mahalanobis_distance = scipy.spatial.distance.mahalanobis(row, mean_vector, inv_cov_matrix)
    mahalanobis_distances.append(mahalanobis_distance)

In [ ]:
training_predictors_df['mahalanobis_dist'] = mahalanobis_distances
training_predictors_df.head()

In [ ]:
#TODO: Need discussion
mahal_threshold = 3 # Typical 'mahal_threshold' is 3
outlier_significance_val = .001

In [ ]:
training_predictors_df['mahal_p_value'] = 1 - scipy.stats.chi2.cdf(training_predictors_df['mahalanobis_dist'], mahal_threshold)

In [ ]:
outlier_condition = (training_predictors_df['mahal_p_value'] < outlier_significance_val)

In [ ]:
nonOutlier_indices = training_predictors_df[~outlier_condition].index

In [ ]:
temp_col = 'response'
training_predictors_df[temp_col] = training_responses

training_predictors_df = training_predictors_df[~outlier_condition]

training_responses = training_predictors_df[temp_col]
training_predictors_df.drop(temp_col, axis=COL, inplace=True)

In [ ]:
training_predictors_df.drop(['mahalanobis_dist', 'mahal_p_value'], axis = COL, inplace=True)
training_predictors_df.reset_index(drop=True)
training_predictors_df.head()

#### Merge features and responses

In [ ]:
train_df = pd.concat([training_predictors_df, pd.DataFrame({consts.RESPONSE_NAME: training_responses})], 
                     axis=COL)
test_df = pd.concat([testing_predictors_df, pd.DataFrame({consts.RESPONSE_NAME: testing_responses})], 
                    axis=COL)

### Output

In [ ]:
out_path = '../data/cleaned_data/'

In [ ]:
out_train_filename = 'training_data.csv'
out_test_filename = 'testing_data.csv'

In [ ]:
train_df.to_csv(out_path + out_train_filename, index=False)
test_df.to_csv(out_path + out_test_filename, index=False)

In [ ]:
%pip freeze > dependencies.txt

In [ ]:

files = helper.get_file_names('20150601') # get filenames from before 6/1/2015
dfs = [pd.read_csv(consts.DATA_PATH_2015 + f"/{f}") for f in files]
main_df = pd.concat(dfs)
main_df.shape

test_df = pd.read_csv(consts.DATA_PATH_2015 + "/data.20150601_1200")

In [ ]:
test_df = pd.read_csv(consts.DATA_PATH_2015 + "/data.20150601_1200")
x_cols = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0","rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0"]
saved_cols = x_cols + [RESPONSE_NAME]
training_df = main_df[saved_cols]
testing_df = test_df[saved_cols]
